# Utilities

In [ ]:
### Importing general modules ...
import healpy as hp
import glob
from scipy.optimize import curve_fit
import pickle
from importlib import reload
import time
import scipy
import os
import numpy as np
import matplotlib.pyplot as plt
import sys
from pylab import arange, show, cm
import gc

### Specific qubic modules
from qubicpack.utilities import Qubic_DataDir
from pysimulators import FitsArray
import pysm
import qubic
from qubic import QubicSkySim as qss
from qubic import fibtools as ft
from qubic import camb_interface as qc
from qubic import SpectroImLib as si
from qubic import mcmc
from qubic import NamasterLib as nam 
from pysimulators.interfaces.healpy import HealpixConvolutionGaussianOperator

reload(qss)
reload(ft)

plt.rc('figure', figsize=(16, 10))
plt.rc('font', size=15)
plt.rcParams['image.cmap'] = 'jet'

In [ ]:
### Some initializations, to be replaced with specific path, or to modify in bash
os.environ['QUBIC_DATADIR'] = '/Users/edgarjaber/myqubic/qubic'
os.environ['QUBIC_DICT'] = '/Users/edgarjaber/myqubic/qubic/dicts'
global_dir = Qubic_DataDir(datafile='instrument.py', datadir=os.environ['QUBIC_DATADIR'])

In [ ]:
### Qubic dictionaries for 150GHz and 220Ghz
config_150, config_220 = 'FI-150', 'FI-220'
dictfilename150 = global_dir + '/doc/FastSimulator/FastSimDemo_{}.dict'.format(config_150)
dictfilename220 = global_dir + '/doc/FastSimulator/FastSimDemo_{}.dict'.format(config_220)
d150, d220 = qubic.qubicdict.qubicDict(), qubic.qubicdict.qubicDict()
d150.read_from_file(dictfilename150)
d220.read_from_file(dictfilename220)
qub_dic = {'150': d150, '220': d220}
center = qubic.equ2gal(d150['RA_center'], d150['DEC_center'])

In [ ]:
def coverage_func(qub_dic, nbands, is_2_band, fq, npoints):
    
    """
    Procedure that returns coverage maps either at both 150 and 220 or only 150 or 220
    
    inputs:
        qub_dic: dict, of the form {'150': qub_dic_150, '220': ...}
        nbands: int, number of subbands
        is_2_band: boolean, if user wants coverage for only 1 frequency band or for 2
        fq: int, either 150 or 220
        npoints: int, number of pointing points
    
    returns:
        q: qubic object
        cov: np.array, coverage map
    """
    
    if is_2_band == True:
        qubic_dict_150, qubic_dict_220 = qub_dic['150'], qub_dic['220']
        qubic_dict_150['nside'], qubic_dict_220['nside'] = 256, 256
        qubic_dict_150['nf_recon'] = nbands
        qubic_dict_220['nf_recon'] = nbands
        qubic_dict_150['nf_sub'] = nbands
        qubic_dict_220['nf_sub'] = nbands
        qubic_dict_150['npointings'] = npoints
        qubic_dict_220['npointings'] = npoints
        p150 = qubic.get_pointing(qubic_dict_150)
        p220 = qubic.get_pointing(qubic_dict_220)
        s150 = qubic.QubicScene(qubic_dict_150)
        s220 = qubic.QubicScene(qubic_dict_220)
        q150 = qubic.QubicMultibandInstrument(qubic_dict_150)
        q220 = qubic.QubicMultibandInstrument(qubic_dict_220)
        _, nus_edge_150, _, _, _, _ = qubic.compute_freq(150, qubic_dict_150['nf_sub'])
        _, nus_edge_220, _, _, _, _ = qubic.compute_freq(220, qubic_dict_220['nf_sub'])
        a150 = qubic.QubicMultibandAcquisition(q150, p150, s150, qubic_dict_150, nus_edge_150)
        a220 = qubic.QubicMultibandAcquisition(q220, p220, s220, qubic_dict_220, nus_edge_220)
        cov1 = a150.get_coverage()
        cov2 = a220.get_coverage()
        a150 = 0
        a220 = 0
        cov1 = np.sum(cov1, axis=0)
        cov2 = np.sum(cov2, axis=0)
        cov1 /= np.max(cov1)
        cov2 /= np.max(cov2)
        return q150, q220, cov1, cov2
    else:
        gc.collect()
        qubic_dict = qub_dic[str(fq)]
        qubic_dict['nside'] = 256
        qubic_dict['nf_recon'] = nbands
        qubic_dict['nf_sub'] = nbands
        qubic_dict['npointings'] = npoints
        p = qubic.get_pointing(qubic_dict)
        s = qubic.QubicScene(qubic_dict)
        q = qubic.QubicMultibandInstrument(qubic_dict)
        _, nus_edge, _, _, _, _ = qubic.compute_freq(fq, qubic_dict['nf_sub'])
        a = qubic.QubicMultibandAcquisition(q, p, s, qubic_dict, nus_edge)
        cov = a.get_coverage()
        a = 0
        cov = np.sum(cov, axis=0)
        cov /= np.max(cov)
        return q, cov

In [ ]:
###Same resolution functions
def get_sigma_iband(q, iband):
    """Get the sigma of one band."""
    sigma = q[iband].synthbeam.peak150.sigma * (150e9 / q[iband].filter.nu)  
    return sigma

def get_sigma_prime(q, iband):
    """Get the sigma needed to convolve one band at the first band resolution."""
    sigma_band0 = get_sigma_iband(q, 0)
    sigma_bandi = get_sigma_iband(q, iband)
    sigma_prime = np.sqrt(sigma_band0**2 - sigma_bandi**2)
    print(sigma_band0, sigma_bandi, sigma_prime)
    return sigma_prime

def make_all_bands_same_resolution(q, maps):
    """ Convolve each band at the first band resolution."""
    nbands = len(q)
    maps_same_reso = np.zeros_like(maps)
    for i in range(nbands):
        sigma_prime = get_sigma_prime(q, i)
        C = HealpixConvolutionGaussianOperator(sigma=sigma_prime)
        maps_same_reso[i] = C(maps[i])
    return maps_same_reso

In [ ]:
### Plot function

def plot_maps(map1, map2, qub_dic, sub_freq, seenpix):
    
    """
    inputs:
        map1, map2: np.array, of the format (nband, npix, nStokes)
        qub_dic: dict, with QSS attributes
        sub_freq: int, at which subfrequency we plot the maps
        
    returns
        matplotlib plot 3x3 of input map, estimated map, difference
    """
    nsig = 3
    Stokes = ['I', 'Q', 'U']
    center = qubic.equ2gal(qub_dic['RA_center'], qub_dic['DEC_center'])
    fig = plt.figure()
    for i in range(3):
        
        sig = np.std(map1[sub_freq, seenpix, i])
        
        hp.gnomview(map1[sub_freq,:,i],
                rot=center,
                reso=15,
                sub=(3,3,3*i+1),
                min=-nsig*sig, max=nsig*sig,
                title= '$\mathbf{s}^{' + Stokes[i] +  '}$')   
      
        hp.gnomview(map2[0,:,i], 
                rot=center,
                reso=15,
                sub=(3,3,3*i+2),
                min=-nsig*sig, max=nsig*sig,
                title='$\hat{\mathbf{s}}^{' + Stokes[i] + '}$')
        
        diff = map1[sub_freq,:,i]/map2[0,:,i]-1
        mm = round(np.mean(diff[seenpix]), 3)
        ss = round(np.std(diff[seenpix]), 3)
        
        hp.gnomview(map2[0,:,i] - map1[sub_freq,:,i],
                rot=center,
                reso=15,
                sub=(3,3,3*i+3),
                min=-nsig*sig, max=nsig*sig,
                title='$\hat{\mathbf{s}}^{' + Stokes[i] +
                    '} - \mathbf{s}^{' + Stokes[i] + '}$ \n mm: ' + str(mm) + ' ss:' + str(ss))
    plt.tight_layout(pad=2.5)

In [ ]:
def histogram_plots(CMB_est, CMB_map, qub_dic, coverage_map):
    
    """
    Function that plots the histograms for 2 maps
    
    inputs: 
        CMB_est: np.array, CMB estimated map
        CMB_map: np.array, CMB simulated map
        qub_dic: dict, qubic dictionary
        coverage_map: np.array, coverage map
        
    returns:
        Plot of histograms for every Stokes parameter with associated RMS
    """
    Stokes = ['I', 'Q', 'U']
    ranges = {'I': [-300, 300], 'Q': [-25, 25], 'U': [-25, 25]}
    seenpix = coverage_map != 0
    fig, axs = plt.subplots(qub_dic['nf_recon'], 3)
    color = ['b', 'r']
    for band in range(qub_dic['nf_recon']):
        for stk in range(3):                                      
            axs[band, stk].hist(CMB_map[band,seenpix,stk],
                                color=color[0],
                                bins=100,
                                range=ranges[Stokes[stk]],
                                alpha=0.75,
                                label='RMS $\mathbf{s}^{'+ Stokes[stk] + '}$ :' + str(round(np.std(CMB_map[band,seenpix,stk]), 3)))
            axs[band, stk].hist(CMB_est[0,seenpix,stk],
                                color=color[1],
                                bins=100,
                                range=ranges[Stokes[stk]],
                                alpha=0.75,
                                label='RMS $\hat{\mathbf{s}}^{'+ Stokes[stk] + '}$ :'  + str(round(np.std(CMB_est[0,seenpix,stk]), 2)))
            axs[0, stk].title.set_text(Stokes[stk] + ' parameter')
            axs[band, stk].legend()
        axs[band, 0].set_ylabel('Band '+ str(band + 1) + '/' + str(qub_dic['nf_recon']))
    #fig.suptitle('Histograms of foregrounds/ILC residuals - 220 $GHz$ ' + str(qub_dic['nf_recon']) + ' recon. bands')
    fig.tight_layout(pad=2.5)

In [ ]:
def scatter_plots(CMB_est, CMB_map, R_nu, qub_dic, coverage, band):
    
    """
    Function that returns the scatter plots of different input maps
    
    inputs:
        CMB_est, CMB_map, R_nu: np.array, corresponding to the different maps
        qub_dic: dict, qubic dictionary
        coverage: np.array, coverage map
        band: int, number of the band
        
    returns:
       scatter plots: CMB_est -- CMB_map, (CMB_map - CMB_est) -- R_nu, for
       every Stokes parameter at the i-th band
    """
    
    Stokes = ['I', 'Q', 'U']
    seenpix = coverage != 0
    fig, axs = plt.subplots(2, 3)
    color = ['b', 'r', 'g']
    for stk in range(3):
        axs[0, stk].scatter(CMB_est[0,seenpix,stk],
                            CMB_map[band,seenpix,stk],
                            color=color[0])
        pval0 = np.polyfit(CMB_est[0,seenpix,stk], CMB_map[band,seenpix,stk], deg=1)
        x = np.linspace(np.min(CMB_est[0,seenpix,stk]), np.max(CMB_est[0,seenpix,stk]), 10)
        axs[0, stk].plot(x, x, color='r', linewidth=5)
        axs[0, stk].plot(x, np.polyval(pval0, x), 'k', linewidth=5)
        
        axs[1, stk].scatter(CMB_est[0,seenpix,stk] - CMB_map[band,seenpix,stk],
                            R_nu[seenpix,stk],
                            color=color[0])
        pval1 = np.polyfit(CMB_est[0,seenpix,stk] - CMB_map[band,seenpix,stk], R_nu[seenpix,stk], deg=1)
        y = np.linspace(np.min(CMB_est[0,seenpix,stk] - CMB_map[band,seenpix,stk]), np.max(CMB_est[0,seenpix,stk] - CMB_map[band,seenpix,stk]), 10)
        axs[1, stk].plot(y, y, color='r', linewidth=5)
        axs[1, stk].plot(y, np.polyval(pval1, y), 'k', linewidth=5)
        
        axs[0, stk].set_ylabel('$\mathbf{s}^{'+ Stokes[stk] + '}$', fontsize=25)
        axs[0, stk].set_xlabel('$\hat{\mathbf{s}}^{'+ Stokes[stk] + '}$', fontsize=25)
        axs[1, stk].set_ylabel('$\mathbf{R}^{'+ Stokes[stk] + '}$', fontsize=25)
        axs[1, stk].set_xlabel('$\hat{\mathbf{s}}^{'+ Stokes[stk] + '} - \mathbf{s}^{'+ Stokes[stk] + '}$', fontsize=25)   

    
    fig.tight_layout(pad=2.5)
        

# Internal Linear Combination (ILC)

There are many map-based astrophysical component separation techniques in the CMB literature, among which is also ILC of which I will now give a quick review.

The maps at a given frequency $\nu$, for every pixel $p$ can be modelled as

$$\mathbf{y}_{\nu}(p) = \mathbf{s}(p) + \mathbf{f}_{\nu}(p) + \mathbf{n}_{\nu}(p)$$

We assume the maps $\mathbf{y}$ are at **the same resolution**.

Given that the CMB signal $\mathbf{s}$ does not depend on the frequency, we can write the following convex estimator:

$$\hat{\mathbf{s}}(p) = \sum_{\nu}\alpha_{\nu}\mathbf{y}_{\nu}(p), \; \; \text{where} \; \; \sum_{\nu}\alpha_{\nu} = 1 $$


The idea is to find the coefficients $\alpha_{\nu}$ such that the variance of the estimator is minimum.

By minimizing the associated Lagrangian, one finds for every frequency $i$:

$$\alpha_{i} = \frac{\sum_{j}[\mathbf{C}^{-1}]_{ij}}{\sum_{kj}[\mathbf{C}^{-1}]_{kj}}$$

where $\mathbf{C} = <\mathbf{yy}^{T}>$ is the covariance matrix of the observations. 

In addition, the variance of the ILC map is given by

$$\sigma^2 = \alpha^{T}\mathbf{C}\alpha$$

In [ ]:
def ILC(noisy_map, coverage, nf):
    
    """
    ILC procedure
    
    inputs:
        noisy_map: np.array, input map of the form (nband, npix, nStokes)
        coverage: np.array, input coverage map
        nf: int, number of bands
        
    returns:
        ILC_est: np.array, CMB estimated map
        weights_list: list, containing the ILC weights for each frequency
        variance_stokes_par: list, containing the variances of the ILC method for each
                             Stokes parameter
    """
    
    def calculate_wi(band, cov):
        
        """
        Calculates the linear combination coefficients of the ILC procedure
        
        inputs:
            band: int, for which frequency band we calculate the weight
            cov: np.array, covariance matrix of the maps
            
        returns:
            weight: float, ILC weight calculated according to the result of Lagrangian minimization 
        """
        
        inv_cov = np.linalg.inv(cov)
        return np.sum(inv_cov, axis=1)[band]/np.sum(inv_cov)
    
    def calculate_var_ILC(weights, cov):
        
        """
        Calculates the variance of ILC estimator
        
        inputs:
            weights: list, containing the ILC weights for each frequency
            cov: np.array, covariance matrix of the maps
            
        returns:
            variance: float, variance of ILC for a given Stokes parameter    
        """
        
        return np.dot(np.transpose(weights), np.dot(cov, weights))
    
    ILC_est = np.empty(noisy_map.shape)
    cov_mats = list(qss.get_cov_nunu(noisy_map, coverage, QUsep=True))
    variance_stokes_par = []
    weights_list = []
    for i in range(3):
        weights = []
        for band in range(nf):
            weights.append(calculate_wi(band, cov_mats[i]))
            ILC_est[0,:,i] += np.multiply(calculate_wi(band, cov_mats[i]), noisy_map[band,:,i])
        variance_stokes_par.append(calculate_var_ILC(weights, cov_mats[i]))
        weights_list.append(weights)
        
    return ILC_est, weights_list, variance_stokes_par

# Study of results at 150GHz

In [ ]:
### Example of how it works ...
q150, cov150 = coverage_func(qub_dic, 3, False, 150, 3000)

In [ ]:
reload(qss)

### QubicSkySim instanciation
seed = 42
sky_config = {'dust': 'd1', 'cmb': seed, 'synchrotron': 's1'}
Qubic_sky_150 = qss.Qubic_sky(sky_config, d150)

### We create a qubic map with JC's fast-simulator
qubic_map_150, noiseless_maps_150, noise150, _ = Qubic_sky_150.get_partial_sky_maps_withnoise(spatial_noise=False,
                                                                                  coverage=cov150)

### Get all maps with same resolution
qubic_map_150 = make_all_bands_same_resolution(q150, qubic_map_150)

In [ ]:
### CMB estimation with ILC
CMB150_est, weights_list, variance_ILC = ILC(qubic_map_150, cov150, 3)

In [ ]:
### Simulated CMB map for comparison
qss_sky_CMB = qss.Qubic_sky({'cmb': seed}, d150)
_, CMB150, _, _ = qss_sky_CMB.get_partial_sky_maps_withnoise(spatial_noise=False, coverage=cov150)

CMB150 = make_all_bands_same_resolution(q150, CMB150)

In [ ]:
seenpix = cov150 != 0
for i in range(3):
    plot_maps(CMB150, CMB150_est, d150, i, seenpix)

## Variance of ILC as a function of the number of subbands

In [ ]:
variance_graph = [[],[],[]]
error_bars = [[],[],[]]
Stokes = ['I','Q','U']
N = 5

for band_num in range(2,6):
    #We define the coverage for the qubic patch, different subbands
    print('Calculating the coverage for ' + str(band_num) + ' bands...')
    d = d150.copy()
    d['nf_sub'] = band_num
    d['nf_recon'] = band_num
    q150, cov150 = coverage_func(qub_dic, band_num, False, 150, 100)
    err_var = [[],[],[]]
    #We do the sky simulation N times
    print('Simulations start...')
    for i in range(N):
        sky_config = {'dust': 'd1', 'cmb': 42, 'synchrotron': 's1'}
        Qubic_sky_150 = qss.Qubic_sky(sky_config, d150)
        qubic_map, _, _, _ = Qubic_sky_150.get_partial_sky_maps_withnoise(spatial_noise=False,
                                                                                     coverage=cov150)
        maps_qubic_same_reso = make_all_bands_same_resolution(q150, qubic_map)
        #We calculate the variance
        for j in range(3):
            err_var[j].append(ILC(maps_qubic_same_reso, cov150, band_num)[1][j])
    print('Appending the variances of ILC...')
    for k in range(3):
        variance_graph[k].append(np.mean(np.asarray(err_var[k])))
        error_bars[k].append(np.var(err_var[k]))

for i in range(3):
    plt.errorbar(range(2,6), variance_graph[i], yerr=error_bars[i], fmt='o', label=Stokes[i] + ' par.')
plt.xlabel('Number of bands')
plt.ylabel(r'$\sigma_{ILC}^2$')
plt.title(r'$\sigma_{ILC}^2$ for I,Q,U Stokes parameters')
plt.legend()
plt.show()

# Histograms, data analysis of the results

In [ ]:
### Histogram plots
histogram_plots(CMB150_est, CMB150, d150, cov150)

The interesting residual for the study of the correlation between the estimated maps and the original maps is for all pixel $p$: 

$$\hat{\mathbf{s}}^{i}(p) - \mathbf{s}^{i}(p) = \sum_{\nu} \alpha_{\nu}^{i}(\mathbf{f}_{\nu}^{i}(p) + \mathbf{n}_{\nu}^{i}(p)) = \mathbf{R}^{i}(p)$$

In [ ]:
R_nu = np.empty(shape=(786432, 3))
for i in range(3):
    for j in range(d220['nf_recon']):
        R_nu[:,i] += weights_list[i][j]*(qubic_map_150[j,:,i] - CMB150[j,:,i]) 

In [ ]:
### Scatter plots
for i in range(3):
    scatter_plots(CMB150_est,CMB150, R_nu, d150, cov150, i)

The bottom line of this study is that this method of component separation seems to work very well with the **I** Stokes parameter and __very bad with the other ones__.

# Hybrid Internal combination with Template Fitting (HILT)

This is a variant of ILC that uses foreground templates to uncover the CMB map. The full description can be found here (following a proposition by Claudia Scoccola): http://www.astronomiaargentina.org.ar/b61a/2019baaa...61a...215L.pdf

For this ILC variant, we need to have foreground templates and also the covariance of the qubic and the foreground maps

## Preliminary: covariance matrixes of Qubic map - Foreground maps, plots...

In [ ]:
sky_config_1 = {'dust': 'd1', 'cmb': 42}
sky_config_2 = {'dust': 'd1'}

Qubic_sky = qss.Qubic_sky(sky_config_1, d150)
FG_sky = qss.Qubic_sky(sky_config_2, d150)

qubic_map, _, _, _ = Qubic_sky.get_partial_sky_maps_withnoise(spatial_noise=False, coverage=cov150)
_, fg_map, _, _ = FG_sky.get_partial_sky_maps_withnoise(spatial_noise=False, coverage=cov150)

### Here we calculate the full covariance matrix
covariance_mat = list(qss.get_cov_nunu(np.asarray(list(qubic_map) + list(fg_map)),
                                       cov150,
                                       QUsep=True))
print(qubic_map.shape)

cmap = cm.bwr
cmap.set_under('w')

### Here we plot the covariance matrixes
plt.figure()
for i in range(3):
    plt.subplot(1,3,i+1)
    plt.imshow(covariance_mat[i][0:5, 5:10], cmap=cmap)
    plt.title('$Cov(QUBIC,FG)$ - ' + Stokes[i])

plt.figure()
for i in range(3):
    plt.subplot(1,3,i+1)
    plt.imshow(covariance_mat[i][5:10, 0:5], cmap=cmap)
    plt.title('$Cov(FG,QUBIC)$ - ' + Stokes[i])
    
plt.figure()
for i in range(3):
    plt.subplot(1,3,i+1)
    plt.imshow(covariance_mat[i][0:5, 0:5], cmap=cmap)
    plt.title('$Cov(QUBIC,QUBIC)$ - ' + Stokes[i])

plt.figure()
for i in range(3):
    plt.subplot(1,3,i+1)
    plt.imshow(covariance_mat[i][5:10, 5:10], cmap=cmap)
    plt.title('$Cov(FG,FG)$ - ' + Stokes[i])
                                                                                                                         

In [ ]:
### We define the different covariance matrixes
A = covariance_mat[0][0:5, 0:5]
B = covariance_mat[0][0:5, 5:10]
C = covariance_mat[0][5:10, 5:10]

### Specific matrixes of the HILT -> look at paper
G_inv = np.linalg.inv(A - np.dot(C, np.dot(np.linalg.inv(B),np.transpose(C))))
H_inv = np.dot(np.linalg.inv(B), np.dot(np.transpose(C), G_inv))

plt.figure()
plt.subplot(1,2,1)
plt.imshow(G_inv, cmap=cmap)
plt.subplot(1,2,2)
plt.imshow(H_inv, cmap=cmap)

# HILT

In [ ]:
def HILT(qubic_map, fg_temp, coverage, nf):

    """
    Hybrid internal linear combination with foreground templates
    
    inputs:
        qubic_map: np.array, qubic map
        fg_temp: np.array, foreground map
        coverage: np.array, coverage map
        nf: int, number of subbands
    
    returns:
        HILT_est: np.array, estimated CMB map
        alpha_w, beta_w: list, with the coefficients of the HILT method
    
    """
    
    def calculate_a_i(band, inv_cov):    
        return np.sum(inv_cov, axis=1)[band]/np.sum(inv_cov)
    
    def calculate_b_i(band, inv_cov1, inv_cov2):
        return np.sum(inv_cov1, axis=1)[band]/np.sum(inv_cov2)
    
    HILT_est = np.empty(qubic_map.shape)
    cov_mats = list(qss.get_cov_nunu(np.asarray(list(qubic_map) + list(fg_temp)), coverage, QUsep=True))
    alpha_w, beta_w = [], [] 
    for i in range(3):
        A, B, C = cov_mats[i][0:nf, 0:nf], cov_mats[i][0:nf, nf:2*nf], cov_mats[i][nf:2*nf, nf:2*nf]
        G_inv = np.linalg.inv(A - np.dot(C, np.dot(np.linalg.inv(B),np.transpose(C))))
        H_inv = np.dot(np.linalg.inv(B), np.dot(np.transpose(C), G_inv))
        alpha_i, beta_i = [], []
        for band in range(nf):
            alpha_i.append(calculate_a_i(band, G_inv))
            beta_i.append(calculate_b_i(band, H_inv, G_inv))
            HILT_est[0,:,i] += (np.multiply(calculate_a_i(band, G_inv), qubic_map[band,:,i]) - 
                                np.multiply(calculate_b_i(band, H_inv, G_inv), fg_temp[band,:,i]))
        alpha_w.append(alpha_i)
        beta_w.append(beta_i)
                
    return HILT_est, alpha_w, beta_w
    

In [ ]:
### Get all maps with same resolution
qubic_map = make_all_bands_same_resolution(q150, qubic_map)
fg_map = make_all_bands_same_resolution(q150, fg_map)


In [ ]:
### Apply the HILT
HILT_est, alpha_i, beta_i = HILT(qubic_map, fg_map, cov150, 5)

In [ ]:
### Simulated CMB map for comparison
qss_sky_CMB = qss.Qubic_sky({'cmb': seed}, d150)
_, CMB150, _, _ = qss_sky_CMB.get_partial_sky_maps_withnoise(spatial_noise=False, coverage=cov150)

CMB150 = make_all_bands_same_resolution(q150, CMB150)

In [ ]:
seenpix = cov150 != 0
for i in range(3):
    plot_maps(CMB150, HILT_est, d150, i, seenpix)